# ATAC-seq Module1: Preprocessing and Quality Control

<!-- <img src="https://storage.googleapis.com/unmc_atac_data_examples/Tutorial1/images/ATACseqWorkflowLesson1.jpg" alt="Drawing" style="width: 1000px;"/> -->

## Overview & Purpose
This short tutorial demonstrates the intial processing steps for ATAC-seq analysis. In this module we focus on generating quality reports of the fastq files, adapter trimming, mapping, and removal of PCR duplicates.

In this tutorial we will process a randomly chosen published dataset. This is available from GEO: GSE67382
Bao X, Rubin AJ, Qu K, Zhang J et al. A novel ATAC-seq approach reveals lineage-specific reinforcement of the open chromatin landscape via cooperation between BAF and p63. Genome Biol 2015 Dec 18;16:284. PMID: 26683334

This dataset is paired-end 50 bp sequencing. We will analyze two samples representing NHEK cells with BAF depletion compared to a control. Note that to allow faster processing we have limited the reads to that of chromosome 4. 
  
### Required Files
In this stage of the module you will use the fastq files that have been prepared. In step1 we will copy these files over to your instance. You can also use this module on your own data or any published ATAC-seq dataset. 

<div class="alert-info" style="font-size:200%">
STEP1: Setup Environment
</div>

Initial items to configure your google cloud environment. In this step we will use conda to install the following packages:

Quality Reporting:
[fastqc](https://anaconda.org/bioconda/fastqc), [multiqc](https://anaconda.org/bioconda/multiqc)

Read Trimming: 
[trimmomatic](https://anaconda.org/bioconda/trimmomatic)

Mapping:
[bowtie2](https://anaconda.org/bioconda/bowtie2)

Deduplication:
[samtools](https://anaconda.org/bioconda/samtools), [picard](https://anaconda.org/bioconda/picard)

In [ ]:
#!python -m ipykernel install --user --name ATACtraining
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
!conda config --prepend channels bioconda
!conda install -y -c bioconda fastqc bowtie2 picard multiqc samtools trimmomatic
!pip install jupyterquiz jupytercards
from jupyterquiz import display_quiz
from IPython.display import IFrame
from IPython.display import display
from jupytercards import display_flashcards
import pandas as pd

## Setup FileSystem
Now lets create some folders to stay organized and copy over our prepared fastq files. We're going to create a directory called "Tutorial1" which we'll use for this module. We'll then create subfolders for our InputFiles and for the files that we'll be creating during this module. We'll also copy over the fasta file for chromosome 4 as well as some bowtie2 index files (don't worry we'll teach you how to create these index files).

In [2]:
#These commands create our directory structure.
#!cd $HOMEDIR
#!mkdir -p Tutorial1
#!mkdir -p Tutorial1/InputFiles
!mkdir -p Tutorial1/QC
!mkdir -p Tutorial1/Trimmed
!mkdir -p Tutorial1/Mapped
#!mkdir -p Tutorial1/RefGenome
#!mkdir -p Tutorial1/LessonImages
#!echo $PWD

#### the google bucket is not working
#These commands help identify the google cloud storage bucket where the example files are held.
# project_id = "nosi-unmc-seq"
# original_bucket = "gs://unmc_atac_data_examples/Tutorial1"
#!gsutil -m cp $original_bucket/images/* Tutorial1/LessonImages
#This command copies our example files to the Tutorial1/Inputfiles folder that we created above.
# ! gsutil -m cp $original_bucket/InputFiles/*fastq.gz Tutorial1/InputFiles
#! gsutil -m cp $original_bucket/InputFiles/hg38* Tutorial1/RefGenome



### OK
Let's make sure that the files copied correctly. You should see 4 files after running the following command:

In [3]:
!ls Tutorial1/InputFiles


CTL_R1.fastq.gz  CTL_R2.fastq.gz  Mutant_R1.fastq.gz  Mutant_R2.fastq.gz


<div class="alert-info" style="font-size:200%">
STEP2: QC
</div>

Sequences are typically provided as files in fastq format. This format includes 4 lines per sequence.


In [6]:
## This file dne
# display_flashcards('Tutorial1/LessonImages/FastqFlashCard.json')

### Click on the above image to see what each line represents.

Next, let's take a look at the sequence quality of the raw reads usinq fastqc:

In [8]:
### check num threads
numthreadsint

7

In [9]:
#This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o Tutorial1/QC Tutorial1/InputFiles/*fastq.gz

#We then use multiqc to summarize the report.
!multiqc -o Tutorial1/QC -f Tutorial1/QC 2> Tutorial1/QC/multiqc_log.txt

#We'll load this into a pandas table to work in this context, but fastqc also produces an html report that you can browse.
dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)

application/gzip
application/gzip
application/gzip
application/gzip


,Sample,Filename,File type,Encoding,Total Sequences,Total Bases,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,...,basic_statistics,per_base_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,CTL_R1,CTL_R1.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,250000.0,12.5 Mbp,0.0,50.0,43.0,52.289537,...,pass,pass,pass,fail,pass,pass,pass,warn,warn,pass
1,CTL_R2,CTL_R2.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,250000.0,12.5 Mbp,0.0,50.0,42.0,51.099795,...,pass,pass,pass,fail,warn,pass,pass,warn,fail,pass
2,Mutant_R1,Mutant_R1.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,250000.0,12.5 Mbp,0.0,50.0,42.0,63.378570,...,pass,pass,pass,fail,pass,pass,pass,warn,warn,pass
3,Mutant_R2,Mutant_R2.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,250000.0,12.5 Mbp,0.0,50.0,42.0,61.583462,...,pass,pass,pass,fail,warn,pass,pass,warn,fail,pass


Alternatively, we can view the fastqc html files:

In [11]:
#We can display the resulting fastqc results.
IFrame(src='Tutorial1/QC/CTL_R1_fastqc.html', width=1080, height=800)

Look at the the "Per base seqeuence content" in the above FastQC report. We'll trim the reads to remove some of this effect. For now, think about possible explanations for this result.

Also look at the "Sequence Duplication Levels". Sometimes duplicates appear due to the PCR amplification step of library preparation. We'll remove duplicates in a later step. 

Lastly, look at the report at the "Overrepresented sequences". What are some possible explanations for this result?

<div class="alert-info" style="font-size:120%">
Trimming
</div>
Next let's trim our sequences.

Why is it particularly important to trim the reads in ATAC-seq? To understand let's review how ATAC-seq works. Tn5 inserts adapter sequences into accessible regions. 

<img src="Tutorial1/LessonImages/adapterinsert.jpg" alt="Drawing" style="width: 400px;"/>

Image source: [Grandi et al., Nature Protocols 2022](https://www.nature.com/articles/s41596-022-00692-9)


What would happen if the distance between inserted sites is short? For example our sequencing lenghth in the example dataset is 50 bp, so what would the sequence look like if our fragment (insert size) is only 30 bp long?  

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 1: Click on the correct answer in following cell.
</div>

In [10]:
# display_quiz("Tutorial1/LessonImages/adapterQuiz.json")

## Let's use trimmomatic to prepare the sequeces before mapping.

In [12]:
#This will trim off N's as well as nextera adapters present in ATAC-seq library preparation. placing the trimmed reads in our Trimmed folder.
!trimmomatic PE -threads $numthreadsint Tutorial1/InputFiles/CTL_R1.fastq.gz Tutorial1/InputFiles/CTL_R2.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz Tutorial1/Trimmed/CTLunpaired_R1.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz Tutorial1/Trimmed/CTLunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3

TrimmomaticPE: Started with arguments:
 -threads 7 Tutorial1/InputFiles/CTL_R1.fastq.gz Tutorial1/InputFiles/CTL_R2.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz Tutorial1/Trimmed/CTLunpaired_R1.fastq.gz Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz Tutorial1/Trimmed/CTLunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 250000 Both Surviving: 249999 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 1 (0.00%) Dropped: 0 (0.00%)
Trim

## Let's do this for the other sample as well.

In [13]:
#This will trim off N's as well as nextera adapters present in ATAC-seq library preparation. placing the trimmed reads in our Trimmed folder.
!trimmomatic PE -threads $numthreadsint Tutorial1/InputFiles/Mutant_R1.fastq.gz Tutorial1/InputFiles/Mutant_R2.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R1.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3

TrimmomaticPE: Started with arguments:
 -threads 7 Tutorial1/InputFiles/Mutant_R1.fastq.gz Tutorial1/InputFiles/Mutant_R2.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R1.fastq.gz Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz Tutorial1/Trimmed/Mutantunpaired_R2.fastq.gz ILLUMINACLIP:Tutorial1/RefGenome/NexteraPE.fa:2:30:10 LEADING:3 TRAILING:3
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 250000 Both Surviving: 249998 (100.00%) Forward Only Surviving: 0 (0.00%) Reverse Only Surviving: 0 (0.00%) Dropp

## Now let's summarize the trimming results.

In [14]:
!fastqc -t $numthreadsint -q -o Tutorial1/Trimmed Tutorial1/Trimmed/*fastq.gz
!multiqc -o Tutorial1/QC -f Tutorial1/Trimmed 2> Tutorial1/QC/multiqc_log.txt

dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_general_stats.txt", sep='\t')
display(dframe)


application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip


,Sample,FastQC_mqc-generalstats-fastqc-percent_duplicates,FastQC_mqc-generalstats-fastqc-percent_gc,FastQC_mqc-generalstats-fastqc-avg_sequence_length,FastQC_mqc-generalstats-fastqc-median_sequence_length,FastQC_mqc-generalstats-fastqc-percent_fails,FastQC_mqc-generalstats-fastqc-total_sequences
0,CTLtrimmed_R1,47.710628,43.0,49.998444,50,10.0,249999.0
1,CTLtrimmed_R2,48.900371,42.0,49.999072,50,20.0,249999.0
2,CTLunpaired_R1,0.000000,0.0,0.000000,0,0.0,0.0
3,CTLunpaired_R2,0.000000,48.0,50.000000,50,30.0,1.0
4,Mutanttrimmed_R1,36.622134,42.0,49.996372,50,10.0,249998.0
5,Mutanttrimmed_R2,38.416652,42.0,49.996400,50,20.0,249998.0
6,Mutantunpaired_R1,0.000000,0.0,0.000000,0,0.0,0.0
7,Mutantunpaired_R2,0.000000,0.0,0.000000,0,0.0,0.0


<div class="alert-info" style="font-size:200%">
Step3: Mapping
</div>
Our fastq files include sequences and quality scores for each base, but we want to figure out which genomic location these sequences came from. To do this we will map each sequence to a reference genome using bowtie2. 
 

Mapping reads requires a reference genome. Due to time and memory considerations, in this tutorial we  prepared that file for you and will only map to chr4. However, in a full analysis, we would map to the entire genome. To do so you would need a fasta file corresponding to the reference genome (e.g. [hg38.fa](https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/)) from which you'd create an index of the genome using bowtie2-build. This can be done with the command: 

bowtie2-build reference_genome_file.fa outputprefix.

As mentioned, we've gone ahead and created the index for you, and, earlier, you copied them into the RefGenome directory. These index files end in the bt2 extension. 



In [15]:
!ls Tutorial1/RefGenome/*bt2

Tutorial1/RefGenome/hg38chr4.1.bt2  Tutorial1/RefGenome/hg38chr4.4.bt2
Tutorial1/RefGenome/hg38chr4.2.bt2  Tutorial1/RefGenome/hg38chr4.rev.1.bt2
Tutorial1/RefGenome/hg38chr4.3.bt2  Tutorial1/RefGenome/hg38chr4.rev.2.bt2


These index files were created from our fasta file:

In [16]:
!ls Tutorial1/RefGenome/*fa

Tutorial1/RefGenome/NexteraPE.fa  Tutorial1/RefGenome/chr4.fa


Notice that the single fasta file created mutiple index files. When we align we'll specify the prefix of the index files.

In [17]:
#Notes: The -x option specifies the prefix of the index. -1 specifies our left-end trimmed reads file. -2 specifies our right-end trimmed reads file. -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x Tutorial1/RefGenome/hg38chr4 -1 Tutorial1/Trimmed/CTLtrimmed_R1.fastq.gz -2 Tutorial1/Trimmed/CTLtrimmed_R2.fastq.gz -S Tutorial1/Mapped/CTL.sam


249999 reads; of these:
  249999 (100.00%) were paired; of these:
    100801 (40.32%) aligned concordantly 0 times
    111601 (44.64%) aligned concordantly exactly 1 time
    37597 (15.04%) aligned concordantly >1 times
    ----
    100801 pairs aligned concordantly 0 times; of these:
      20903 (20.74%) aligned discordantly 1 time
    ----
    79898 pairs aligned 0 times concordantly or discordantly; of these:
      159796 mates make up the pairs; of these:
        143536 (89.82%) aligned 0 times
        6881 (4.31%) aligned exactly 1 time
        9379 (5.87%) aligned >1 times
71.29% overall alignment rate


In [18]:
##Let's do the same thing for our other sample.
!bowtie2 -p $numthreadsint -x Tutorial1/RefGenome/hg38chr4 -1 Tutorial1/Trimmed/Mutanttrimmed_R1.fastq.gz -2 Tutorial1/Trimmed/Mutanttrimmed_R2.fastq.gz -S Tutorial1/Mapped/Mutant.sam

249998 reads; of these:
  249998 (100.00%) were paired; of these:
    75447 (30.18%) aligned concordantly 0 times
    127648 (51.06%) aligned concordantly exactly 1 time
    46903 (18.76%) aligned concordantly >1 times
    ----
    75447 pairs aligned concordantly 0 times; of these:
      21698 (28.76%) aligned discordantly 1 time
    ----
    53749 pairs aligned 0 times concordantly or discordantly; of these:
      107498 mates make up the pairs; of these:
        90592 (84.27%) aligned 0 times
        5712 (5.31%) aligned exactly 1 time
        11194 (10.41%) aligned >1 times
81.88% overall alignment rate


### Answer the following question only if you are using the example dataset we provided. This question is simply a check to ensure everything was processed correctly.

In [20]:
# display_flashcards('Tutorial1/LessonImages/alignment.json')

## Bowtie2 output a file in [sam format](https://samtools.github.io/hts-specs/SAMv1.pdf) which contains the original sequence, quality scores, and the genomic coordinates matching each read. 

In the next commands we'll convert the file to the more compressed [bam format](https://genome.ucsc.edu/goldenPath/help/bam.html) and sort the reads by chromosomal coordinates.

In [21]:
#This will convert to bam by using samtools view with the -b option. The h and S option tells samtools that the file has a header and is in sam format. We will pipe this to samtools sort. Pay attention to the "-" at the end of the sort command which tells samtools to use stdin.
!samtools view -q 10 -bhS Tutorial1/Mapped/CTL.sam | samtools sort -o Tutorial1/Mapped/CTL.bam - 
print("done")

done


In [22]:
#Let's do the same thing for our Mutant sample.
!samtools view -q 10 -bhS Tutorial1/Mapped/Mutant.sam | samtools sort -o Tutorial1/Mapped/Mutant.bam - 
print("done")

done


You may have noticed the parameters -bhS and -q 10 in the above commands. Briefly, -bhS describes aspects of the file to samtools, such that you want to output a bam file (the b option), that it has a header (the h option), and that it is currently in sam format (the S option). We also specified -q 10 which removes reads with a mapping score <= 10. 

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 2: Click on the correct answer in the following cell.
</div>

In [24]:
# display_quiz("Tutorial1/LessonImages/mappingquality.json")

<div class="alert-info" style="font-size:200%">
Step4: Removal of Duplicates
</div>
It's important to remove duplicates from our reads because part of the ATAC-seq method includes a PCR step for library amplification. This can create biases in the data resulting from PCR duplicates. To understand how PCR duplicates can affect the analysis, let's jump ahead a bit. Accessibile sites are represented by ATAC-seq "peaks" of signal.

<img src="Tutorial1/LessonImages/PeaksExample.jpg" alt="Drawing" style="width: 200px;"/>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 3: Click on the correct answer in the following cell.
</div>

In [25]:
display_quiz("Tutorial1/LessonImages/duplicateQuiz.json")

<IPython.core.display.Javascript object>

Okay, let's remove these duplicates using Picard.

In [26]:
#this will take the sorted bam file and remove duplicates, saving a new bam file and a summary in a text file.
!picard MarkDuplicates --REMOVE_DUPLICATES TRUE -I Tutorial1/Mapped/CTL.bam -O Tutorial1/Mapped/CTL_dedup.bam --METRICS_FILE Tutorial1/Mapped/CTL_dedup_metrics.txt --QUIET 2> Tutorial1/Mapped/PicardLog.txt
print("done")

done


In [27]:
#We also should do this for the other sample.
!picard MarkDuplicates --REMOVE_DUPLICATES TRUE -I Tutorial1/Mapped/Mutant.bam -O Tutorial1/Mapped/Mutant_dedup.bam --METRICS_FILE Tutorial1/Mapped/Mutant_dedup_metrics.txt --QUIET 2> Tutorial1/Mapped/PicardLog2.txt
print("done")

done


In [28]:
#We can use multiqc to summarize the metrics
!multiqc -o Tutorial1/QC -f Tutorial1/Mapped 2> Tutorial1/Mapped/multiqc_log.txt
dframe = pd.read_csv("Tutorial1/QC/multiqc_data/multiqc_general_stats.txt", sep='\t')
display(dframe)

,Sample,Picard_mqc-generalstats-picard-PERCENT_DUPLICATION
0,CTL,0.297989
1,Mutant,0.275425


<div class="alert-success" style="font-size:200%">
Great job! 
</div>
We have completed the preprocessing steps and are ready to move on to some downstream analysis. Take a break here or move on to the next tutorial: 

[Visualization and Peak Detection](./ATACseq_Tutorial2_PeakDetection.ipynb). 
